<a href="https://colab.research.google.com/github/ninadcpa/DSC540/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSC540 - Data Wrangling - Final Project

## Title - Public Justice System Expenditures and Crime Control effectiveness across US States

@author Ninad Patkhedkar  
@guide  Catherine Williams

Term - Spring 2021  
Bellevue Univeristy 

---



# Milesstone 1

# Introduction
> Effective Law and Order is one of the most important necessity of society. It has direct effect on living standard and crime rate in that area. The whole "Law and Order" aka Justice system consists of multiple functions which performs duties as below   

> **Police Protection Services** - law enforcement, patrolling, traffic safety, parking meter read, animal warden etc. 

> **Judicial and Legal Services** - civil and criminal functions of courts, state’s attorneys, court reporters, register of wills etc. 

> **Correction Services** - prisons, reformatories, rehabilitation centers, parole boards, pardon boards etc. 

> To effectively govern any area, all these functions need resources i.e. people to perform duties and funding.

In this project I will use public dataset about state wise employment and expenditures on various functions. I will also correlate state wise crime during same period. I will investigate and find more information about how effectively Justice and Legal system is managed in different states.  I will also try to find any correlation between population of state, expenses and number of employees etc. 

I will use 3 different sources of data 

# Dataset Sources  

1. **Flat file**  - jeee16t03.csv
Public dataset for fiscal 2016 from Bureau of Justice Statistics on Justice Expenditure - https://www.bjs.gov/content/pub/sheets/jeee16p.zip 

2. **Web Site**  
Public dataset on Crime for fiscal 2016 from FBI - https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-3

3. **API** 
Public dataset for fiscal 2016 from Bureau of Justice Statistics on Employee Extracts - https://data.world/usbjs/justice-expenditure-and-employment-2016/workspace/file?filename=jeee16t08.csv

API Data is from same source as #1 i.e. Bureau of Justice Statistics. As requiremnet is to fetch data over API, the dataset is maintained on ***data.world***  and will be fetched in json form using ***requests*** library.



```
# sample linux command to fetch data in json format
curl --request POST \
  --url "https://api.data.world/v0/sql/ninadcpa/dsc540?includeTableSchema=true" \
  --header "Authorization: Bearer <api_token>" \
  --header "Accept: application/json" \
  --data-urlencode 'query=SELECT * FROM jeee16t08'
```

For Definitions, Methodology and other information, refer detailed guide -  
https://www.bjs.gov/content/pub/pdf/jeeeguide.pdf  
https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-3/table-3-data-declaration.pdf

# Data Prep

All 3 datasets have common key of State name. It has one-to-one relationship across all 3 datasets.
 
I will cleanup each data set and and create a consolidated dataset by "State" in this section. Here are few things I will perform as part of cleanup   

* **Simplify column names by converting to lower case and spaces replaced by underscores**
* **Rename long column names to short names e.g. population_2016_thousands to population_k**
* **Remove records of Total federal govt, Local city govt, Municipality govt data and keep only State data**
* **Remove records where population is not present**
* **Trim State abbreviations from records i.e. Virginia (VA) -> Virginia** 
* **Join Datasets by using "State" as common key**
  
# Objective
As part of this Project, I will try to find answer following questions​
1. How state population affects overall cost and per capita cost?
2. How number of employees in police protection services affect the cost of police protection? 
3. How number of employees in judicial and legal services affect the cost of judicial and legal functions?
4. How number of employees in correction services affect the cost of correction functions?
5. Which states are doing great job with overall crime control?
6. What is relation between number of employees in police protection services and crime in the area?


In [2]:
# This code is written for mounting code folder checked out from github on google drive
# All project code will be maintained inside folder "DSC540"

from google.colab import drive
import sys
import os

drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DSC540')
os.chdir('/content/drive/MyDrive/Colab Notebooks/DSC540')


Mounted at /content/drive


# Milestone 2

Cleaning/Formatting Flat File Source
Perform at least 5 data transformation and/or cleansing steps to your flat file data. For example:

> Replace Headers  
Format data into a more readable format  
Identify outliers and bad data  
Find duplicates  
Fix casing or inconsistent values  
Conduct Fuzzy Matching

---

I am using flat file in csv format "jeee16t03.csv" 

Lets load the file data in panda dataframe and check some records

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading csv file into panda dataframe
df_jee03 = pd.read_csv("jeee16t03.csv")
df_jee03.head(10)

,State and type of government,Population 2016 thousands,Total direct expenditure,total justice system Amount,total justice system percent,Police protection Amount,police protection Percent,judician and legal Amount,judicial and legal Percent,corrections Amount,corrections Percent
0,"States-local, total",323071,3514583394,233483760,6.6,109210078,46.8,46256957,19.8,78016725,33.4
1,State,-,1692408177,86868791,5.1,15003016,17.3,22825679,26.3,49040096,56.5
2,"Local, total",-,1822175217,146614969,8.0,94207062,64.3,23431278,16.0,28976629,19.8
3,Counties,-,394148057,71684805,18.2,29596808,41.3,17767223,24.8,24320774,33.9
4,Municipalities,-,583324970,69058355,11.8,59702889,86.5,5274743,7.6,4080723,5.9
5,Alabama (AL),4865,45277563,2335599,5.2,1251270,53.6,362060,15.5,722269,30.9
6,AL State,-,23075269,870057,3.8,165168,19.0,200146,23.0,504743,58.0
7,"AL Local, total",-,22202294,1465542,6.6,1086102,74.1,161914,11.0,217526,14.8
8,AL Counties,-,2244358,644571,28.7,358334,55.6,90688,14.1,195549,30.3
9,AL Municipalities,-,6596181,820971,12.4,727768,88.6,71226,8.7,21977,2.7


In [4]:
df_jee03.shape

(256, 11)

Data contains 256 observationas and 11 variables.  
Dataset contains records for fedral, state, local governments, muncipalties and counties. 

For this project, as state is common key field for all 3 datasets. Hence I will try to filter records only at state level.

I will perform as part of cleanup  

* **Simplify column names by converting to lower case and spaces replaced by underscores**
* **Rename long column names to short names e.g. population_2016_thousands to population_k**
* **Remove records of Total federal govt, Local city govt, Municipality govt data and keep only State data**
* **Remove records where population is not present**
* **Trim State abbreviations from records i.e. Virginia (VA) -> Virginia** 

In [5]:
# remove spaces in columns name and replace with underscores
df_jee03.columns = df_jee03.columns.str.replace(' ','_')
df_jee03.columns

Index(['State_and_type_of_government', 'Population_2016_thousands',
       'Total_direct_expenditure', 'total_justice_system_Amount',
       'total_justice_system_percent', 'Police_protection_Amount',
       'police_protection_Percent_', 'judician_and_legal_Amount',
       'judicial_and_legal_Percent', 'corrections_Amount',
       'corrections_Percent'],
      dtype='object')

In [6]:
# changing all colunmn names to lower case
df_jee03.rename(columns=str.lower, inplace=True)
df_jee03.columns

Index(['state_and_type_of_government', 'population_2016_thousands',
       'total_direct_expenditure', 'total_justice_system_amount',
       'total_justice_system_percent', 'police_protection_amount',
       'police_protection_percent_', 'judician_and_legal_amount',
       'judicial_and_legal_percent', 'corrections_amount',
       'corrections_percent'],
      dtype='object')

In [7]:
# replace long column names to shorter ones
df_jee03.rename(columns={"state_and_type_of_government" : "state", "population_2016_thousands": "population_k"},inplace=True)
df_jee03.columns

Index(['state', 'population_k', 'total_direct_expenditure',
       'total_justice_system_amount', 'total_justice_system_percent',
       'police_protection_amount', 'police_protection_percent_',
       'judician_and_legal_amount', 'judicial_and_legal_percent',
       'corrections_amount', 'corrections_percent'],
      dtype='object')

One observation is that, population field is not populated for local, county or muncipalty governments. Hence if records with population value as empty ( "-" in this case) are dropped, we can automatically get only state level records.

In [8]:
# drop records where population value is missing
df_jee03 = df_jee03[df_jee03['population_k'] != '-']
df_jee03.head(10)

,state,population_k,total_direct_expenditure,total_justice_system_amount,total_justice_system_percent,police_protection_amount,police_protection_percent_,judician_and_legal_amount,judicial_and_legal_percent,corrections_amount,corrections_percent
0,"States-local, total",323071,3514583394,233483760,6.6,109210078,46.8,46256957,19.8,78016725,33.4
5,Alabama (AL),4865,45277563,2335599,5.2,1251270,53.6,362060,15.5,722269,30.9
10,Alaska (AK),742,15808697,962214,6.1,370209,38.5,254000,26.4,338005,35.1
15,Arizona (AZ),6945,58975013,4929687,8.4,2261558,45.9,983419,19.9,1684710,34.2
20,Arkansas (AR),2990,27299957,1507133,5.5,691059,45.9,220343,14.6,595731,39.5
25,California (CA),39209,532948138,41714177,7.8,17570133,42.1,8675761,20.8,15468283,37.1
30,Colorado (CO),5541,57293994,3940585,6.9,1873320,47.5,754162,19.1,1313103,33.3
35,Connecticut (CT),3579,45649898,2748059,6.0,1236997,45.0,826903,30.1,684159,24.9
39,Delaware (DE),949,11413711,864358,7.6,348027,40.3,207990,24.1,308341,35.7
44,District of Columbia,687,16593661,870775,5.2,593345,68.1,132515,15.2,144915,16.6


In [9]:
# Check if any of the column contains missing value NaN
df_jee03.isna().sum(axis=0)

state                           0
population_k                    0
total_direct_expenditure        0
total_justice_system_amount     0
total_justice_system_percent    0
police_protection_amount        0
police_protection_percent_      0
judician_and_legal_amount       0
judicial_and_legal_percent      0
corrections_amount              0
corrections_percent             0
dtype: int64

In [18]:
# Check if any of the column contains NULL value
df_jee03.isnull().sum(axis=0)

state                           0
population_k                    0
total_direct_expenditure        0
total_justice_system_amount     0
total_justice_system_percent    0
police_protection_amount        0
police_protection_percent_      0
judician_and_legal_amount       0
judicial_and_legal_percent      0
corrections_amount              0
corrections_percent             0
dtype: int64

None of the column containes any missing or null value.

In [10]:
df_jee03.shape

(52, 11)

In [11]:
# Reset index from 0 as records are dropped.
df_jee03.reset_index(drop=True, inplace=True)
df_jee03.head(10)

,state,population_k,total_direct_expenditure,total_justice_system_amount,total_justice_system_percent,police_protection_amount,police_protection_percent_,judician_and_legal_amount,judicial_and_legal_percent,corrections_amount,corrections_percent
0,"States-local, total",323071,3514583394,233483760,6.6,109210078,46.8,46256957,19.8,78016725,33.4
1,Alabama (AL),4865,45277563,2335599,5.2,1251270,53.6,362060,15.5,722269,30.9
2,Alaska (AK),742,15808697,962214,6.1,370209,38.5,254000,26.4,338005,35.1
3,Arizona (AZ),6945,58975013,4929687,8.4,2261558,45.9,983419,19.9,1684710,34.2
4,Arkansas (AR),2990,27299957,1507133,5.5,691059,45.9,220343,14.6,595731,39.5
5,California (CA),39209,532948138,41714177,7.8,17570133,42.1,8675761,20.8,15468283,37.1
6,Colorado (CO),5541,57293994,3940585,6.9,1873320,47.5,754162,19.1,1313103,33.3
7,Connecticut (CT),3579,45649898,2748059,6.0,1236997,45.0,826903,30.1,684159,24.9
8,Delaware (DE),949,11413711,864358,7.6,348027,40.3,207990,24.1,308341,35.7
9,District of Columbia,687,16593661,870775,5.2,593345,68.1,132515,15.2,144915,16.6


In [12]:
# delete first index as its not state level record
df_jee03.drop(index=0,inplace=True)
df_jee03.head(10)

,state,population_k,total_direct_expenditure,total_justice_system_amount,total_justice_system_percent,police_protection_amount,police_protection_percent_,judician_and_legal_amount,judicial_and_legal_percent,corrections_amount,corrections_percent
1,Alabama (AL),4865,45277563,2335599,5.2,1251270,53.6,362060,15.5,722269,30.9
2,Alaska (AK),742,15808697,962214,6.1,370209,38.5,254000,26.4,338005,35.1
3,Arizona (AZ),6945,58975013,4929687,8.4,2261558,45.9,983419,19.9,1684710,34.2
4,Arkansas (AR),2990,27299957,1507133,5.5,691059,45.9,220343,14.6,595731,39.5
5,California (CA),39209,532948138,41714177,7.8,17570133,42.1,8675761,20.8,15468283,37.1
6,Colorado (CO),5541,57293994,3940585,6.9,1873320,47.5,754162,19.1,1313103,33.3
7,Connecticut (CT),3579,45649898,2748059,6.0,1236997,45.0,826903,30.1,684159,24.9
8,Delaware (DE),949,11413711,864358,7.6,348027,40.3,207990,24.1,308341,35.7
9,District of Columbia,687,16593661,870775,5.2,593345,68.1,132515,15.2,144915,16.6
10,Florida (FL),20630,167229459,14463341,8.6,7848477,54.3,2366274,16.4,4248590,29.4


In [13]:
# check the data type of each field
df_jee03.dtypes

state                            object
population_k                     object
total_direct_expenditure          int64
total_justice_system_amount       int64
total_justice_system_percent    float64
police_protection_amount          int64
police_protection_percent_      float64
judician_and_legal_amount         int64
judicial_and_legal_percent      float64
corrections_amount                int64
corrections_percent             float64
dtype: object

In [14]:
# coverting population_k to numeric type
df_jee03['population_k'] = df_jee03['population_k'].apply(pd.to_numeric)
df_jee03.dtypes

state                            object
population_k                      int64
total_direct_expenditure          int64
total_justice_system_amount       int64
total_justice_system_percent    float64
police_protection_amount          int64
police_protection_percent_      float64
judician_and_legal_amount         int64
judicial_and_legal_percent      float64
corrections_amount                int64
corrections_percent             float64
dtype: object

In [15]:
import re
# Remove any any text starting from parenthisis "("" to end using regex
df_jee03['state'] = df_jee03['state'].apply(lambda x: re.sub("\\(.*", "",x))

In [16]:
df_jee03['state'].head()

1       Alabama 
2        Alaska 
3       Arizona 
4      Arkansas 
5    California 
Name: state, dtype: object

In [17]:
# Removing any leading and trailing spaces and coverting state names to uppercase
df_jee03['state'] = df_jee03['state'].str.strip()
df_jee03['state'] = df_jee03['state'].str.upper()
df_jee03['state'].head()

1       ALABAMA
2        ALASKA
3       ARIZONA
4      ARKANSAS
5    CALIFORNIA
Name: state, dtype: object